In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import ipywidgets as widgets
%gui asyncio
import asyncio

In [2]:
url="https://raw.githubusercontent.com/alooperalta/movieRecommendation/master/content/ml-latest-small/movies.csv"
movies=pd.read_csv(url)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movie_title=movies['title']
Genres = movies['genres'].str.get_dummies(sep='|')
movies = pd.concat([movies, Genres], axis=1)
movies.drop(['title','genres','(no genres listed)'],axis=1,inplace=True)
movies.head()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Model Creation

In [4]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import random

In [5]:
X=movies
n_clusters=170
recommender=KMeans(n_clusters=n_clusters,random_state=1)
labels=recommender.fit_predict(X)

In [6]:
rndi=dict()
for i in range(n_clusters):
    rndi[i]=[]
for i in range(len(labels)):
    rndi[labels[i]].append(movie_title[i])

In [7]:
cnt=0
for x in rndi.values():
    if len(x)<6:
        cnt+=1
cnt

1

In [8]:
from IPython.display import clear_output
x=input()
clear_output(wait=True)
ind=movie_title[movie_title==x].index[0]
choice=random.sample(rndi[labels[ind]],5)
for i in choice:
    print(i)

Immortal Beloved (1994)
Boys of St. Vincent, The (1992)
Nobody's Fool (1994)
Nine Months (1995)
Outbreak (1995)


In [27]:
def getRecom(x):
    ind=movie_title[movie_title==x].index[0]
    choice=random.sample(rndi[labels[ind]],5)
    print('\nFinding Recommendations Similar to', x,'...')
    time.sleep(2)
    print('\nTop Matches for',x, 'are:\n' )
    for i in choice:
        if (i==x):
            print(random.sample(rndi[labels[ind]],1)[0]) 
        else:
            print(i)
    print("")

def model():        
        o = movie_title
        opt = pd.concat([pd.Series(['Select an Option']),o])
        dropdown = widgets.Dropdown(
            options= opt,
            description='Chose your favourite Movie:',
            disabled=False,
        )
        btn = widgets.Button(description='Find recommended movies')
        display(dropdown)
        display(btn)
        def btn_eventhandler(obj):
            if(dropdown.value!='Select an Option'):
                getRecom(dropdown.value)
        btn.on_click(btn_eventhandler)

In [24]:
model()

Dropdown(description='Chose your favourite Movie:', options=('Select an Option', 'Toy Story (1995)', 'Jumanji …

Button(description='Find recommended movies', style=ButtonStyle())

Finding Recommendations Similar to Gintama: The Final Chapter - Be Forever Yorozuya (2013) ...

Top Matches for Gintama: The Final Chapter - Be Forever Yorozuya (2013) are:

My Scientology Movie (2016)
Karate Bullfighter (1975)
Buck Rogers in the 25th Century (1979)
Alice Through the Looking Glass (2016)
Kicking Off (2016)

Finding Recommendations Similar to Kicking Off (2016) ...

Top Matches for Kicking Off (2016) are:

The Crew (2016)
Captain Fantastic (2016)
The Handmaiden (2016)
['The Crew (2016)']
The Shallows (2016)


Finding Recommendations Similar to Kicking Off (2016) ...


Top Matches for Kicking Off (2016) are:

The BFG (2016)
SORI: Voice from the Heart (2016)
The Crew (2016)
Toni Erdmann (2016)
Gintama: The Final Chapter - Be Forever Yorozuya (2013)


Finding Recommendations Similar to Kicking Off (2016) ...


Top Matches for Kicking Off (2016) are:

Kill Command (2016)
World of Glory (1991)
The Nice Guys (2016)
Gen-X Cops (1999)
My Scientology Movie (2016)


Finding Recom

In [29]:
import pickle
filename = 'recommender.pkl'
pickle_out = open(filename, 'wb')
pickle.dump(recommender, pickle_out)
pickle_out.close()

In [32]:
import pickle
filename = 'recommender.pkl'
pickle_in = open(filename, 'rb')
loaded_model = pickle.load(pickle_in)
loaded_model

KMeans(n_clusters=170, random_state=1)